In [1]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install tensorflow keras opencv-python pandas scikit-learn kaggle

In [3]:
import os
# Membuat folder .kaggle dan memindahkan file kaggle.json ke folder tersebut
os.makedirs('/root/.kaggle', exist_ok=True)
# Ganti dengan path ke file kaggle.json yang telah diunggah ke Colab
!cp /content/kaggle.json /root/.kaggle/

# Mengatur permission file agar dapat digunakan
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
# Mengunduh dataset FER2013 dari Kaggle
!kaggle datasets download -d msambare/fer2013

In [5]:
# Mengekstrak file ZIP dataset
!unzip fer2013.zip -d /content/fer2013

In [6]:
import pandas as pd
import numpy as np
import cv2

# Menyiapkan data untuk training
X = []
y = []
emotion_dict = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy', 4: 'neutral', 5: 'sad', 6: 'surprise'}

# Menambahkan data gambar dan label
image_paths = ['/content/fer2013/images/'+file for file in os.listdir('/content/fer2013/images/')]
for image_path in image_paths:
    # Membaca gambar
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (48, 48))
    gray = np.expand_dims(gray, axis=-1)  # Menambahkan channel grayscale
    gray = gray / 255.0  # Normalisasi gambar
    X.append(gray)

# Membaca label dari file CSV
labels = pd.read_csv('/content/fer2013/fer2013.csv')
for index, row in labels.iterrows():
    y.append(row['emotion'])

X = np.array(X)
y = np.array(y)

# Membagi data menjadi data latih dan data uji
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Data Training: {X_train.shape}, Data Testing: {X_test.shape}")

In [7]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Membuat model CNN untuk deteksi ekspresi wajah
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(7, activation='softmax')  # 7 kelas ekspresi (Angry, Disgust, Fear, Happy, Sad, Surprise, Neutral)
])

# Menyusun model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Melihat ringkasan model
model.summary()

In [8]:
# Melatih model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Menyimpan model setelah pelatihan selesai
model.save('/content/drive/MyDrive/face_analysis_project/models/emotion_model.h5')

In [9]:
import matplotlib.pyplot as plt

# Plot akurasi training dan validasi
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss training dan validasi
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [10]:
# Model disimpan di Google Drive setelah pelatihan selesai
model.save('/content/drive/MyDrive/face_analysis_project/models/emotion_model.h5')

In [11]:
# Menggunakan model untuk memprediksi ekspresi wajah pada gambar baru
def predict_expression(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Mengubah gambar ke grayscale
    image = cv2.resize(image, (48, 48))  # Ubah ukuran gambar menjadi 48x48
    image = image.reshape(1, 48, 48, 1)  # Ubah bentuknya menjadi (1, 48, 48, 1)
    image = image / 255.0  # Normalisasi
    prediction = model.predict(image)
    emotion_index = np.argmax(prediction)  # Mengambil indeks kelas dengan probabilitas tertinggi
    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    return emotions[emotion_index]

# Contoh penggunaan
img = cv2.imread('/content/drive/MyDrive/face_analysis_project/data/fer2013/images/sample_image.jpg')
expression = predict_expression(img)
print(f"Predicted Expression: {expression}")